In [197]:
import theano.tensor as T
from theano import shared, function, config
import numpy as np
from theano import gradient
from theano import grad
from theano.tensor import nlinalg

In [198]:
length=3

z = T.dvector()

alpha = shared(0, "alpha")
beta = shared(5, "beta")
z0 = shared(np.zeros(length), "z0")

r = (z - z0).norm(2)
# equals: r = T.sqrt(((z - z0)**2).sum())  # i.e. correct norm
h = 1 / (alpha + r)
f = z + beta*h*(z - z0)

parameters = [alpha, beta, z0]

In [210]:
z = T.dvector()
T.tanh(z)

Elemwise{tanh,no_inplace}.0

In [218]:
a = T.dscalar(name="a")

In [228]:
inner = 2*a
h = (inner) **2

In [229]:
dh = grad(h, inner) # = 2 *(2*a)

In [223]:
theano.pp(dh)

'(((fill(((TensorConstant{2} * a) ** TensorConstant{2}), TensorConstant{1.0}) * TensorConstant{2}) * ((TensorConstant{2} * a) ** (TensorConstant{2} - TensorConstant{1}))) * TensorConstant{2})'

In [230]:
f = function([a], [h, dh])

In [231]:
f(3)

[array(36.0), array(12.0)]

In [209]:
dtanh(z)

TypeError: 'list' object is not callable

In [199]:
dh = grad(h, r)

In [200]:
_f = function([z], [h, dh])
_f(range(3))

[array(0.4472135954999579), array(-0.19999999999999996)]

In [181]:
jac = gradient.jacobian(f, z)
jac_det = nlinalg.Det()(jac)

In [182]:
fjac_det = function([z], jac_det)

In [183]:
num_z = np.arange(length)
fjac_det(num_z)

array(10.472135954999583)

In [194]:
%timeit fjac_det(num_z)

10000 loops, best of 3: 124 µs per loop


In [190]:
_alpha, _beta, _z0 = alpha.get_value(), beta.get_value(), z0.get_value()

def r(z):
    return np.linalg.norm(z - _z0)

def h(r):
    return 1/(_alpha + r)

def dh(r):
    return -1/(_alpha + r)**2

def f(z):
    return z + _beta * h(_alpha, r(z))*(z-_z0)   # equation (14,a)

def finv(z_):
    pass  # TODO

def det(z):
    return (1 + _beta*h(r(z)))**(length-1) * (1 + _beta * h(r(z)) + _beta*dh(r(z))*r(z))

In [191]:
_alpha, _beta, _z0

(array(0), array(5), array([ 0.,  0.,  0.]))

In [192]:
det(num_z)

10.472135954999585

In [193]:
%timeit det(num_z)

The slowest run took 7.48 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 22.2 µs per loop


In [101]:
np.linalg.norm(num_z -_z0), np.sqrt(((num_z - _z0)**2).sum())

(3.7416573867739413, 3.7416573867739413)